In [ ]:
# AVALIAÇÃO 1 - PROGRAMAÇÃO DINÂMICA ESTOCÁSTICA
import numpy as np

# Estados e ações do ambiente
estados = ['s1', 's2']
acoes = {'s1': ['x11', 'x12'], 's2': ['x21', 'x22']}

# Recompensa para cada estado e ação
def recompensa(estado, acao):
    if estado == 's1':
        return 1
    else:
        return 0

# Probabilidades de transição (estado, ação -> próximo estado)
prob_transicao = {
    ('s1', 'x11'): {'s1': 0.5, 's2': 0.5},
    ('s1', 'x12'): {'s1': 0.7, 's2': 0.3},
    ('s2', 'x21'): {'s1': 0.1, 's2': 0.9},
    ('s2', 'x22'): {'s1': 0.9, 's2': 0.1}
}

# Fator de desconto e limite de convergência
fator_desconto = 0.9
limite_convergencia = 0.01

# (a) Iteração de Valor
def iteracao_valor():
    valores = {estado: 0.0 for estado in estados}
    while True:
        delta = 0
        valores_novos = valores.copy()
        for estado in estados:
            melhores_valores_acao = []
            for acao in acoes[estado]:
                valor_acao = recompensa(estado, acao) + fator_desconto * sum(
                    prob_transicao[(estado, acao)][prox_estado] * valores[prox_estado]
                    for prox_estado in estados
                )
                melhores_valores_acao.append(valor_acao)
            valores_novos[estado] = max(melhores_valores_acao)
            delta = max(delta, abs(valores_novos[estado] - valores[estado]))
        valores = valores_novos
        if delta < limite_convergencia:
            break
    return valores

# (b) Política ótima a partir dos valores
def obter_politica_gulosa(valores):
    politica = {}
    for estado in estados:
        melhor_acao = None
        melhor_valor = -float('inf')
        for acao in acoes[estado]:
            valor_acao = recompensa(estado, acao) + fator_desconto * sum(
                prob_transicao[(estado, acao)][prox_estado] * valores[prox_estado]
                for prox_estado in estados
            )
            if valor_acao > melhor_valor:
                melhor_valor = valor_acao
                melhor_acao = acao
        politica[estado] = melhor_acao
    return politica

# (c) Iteração de Política
def iteracao_politica():
    politica = {estado: acoes[estado][0] for estado in estados}
    valores = {estado: 0.0 for estado in estados}
    estavel = False
    while not estavel:
        # Avaliação
        while True:
            delta = 0
            valores_novos = valores.copy()
            for estado in estados:
                acao = politica[estado]
                valores_novos[estado] = recompensa(estado, acao) + fator_desconto * sum(
                    prob_transicao[(estado, acao)][prox_estado] * valores[prox_estado]
                    for prox_estado in estados
                )
                delta = max(delta, abs(valores_novos[estado] - valores[estado]))
            valores = valores_novos
            if delta < limite_convergencia:
                break
        # Melhoria
        estavel = True
        for estado in estados:
            acao_antiga = politica[estado]
            melhor_acao = acao_antiga
            melhor_valor = -float('inf')
            for acao in acoes[estado]:
                valor_acao = recompensa(estado, acao) + fator_desconto * sum(
                    prob_transicao[(estado, acao)][prox_estado] * valores[prox_estado]
                    for prox_estado in estados
                )
                if valor_acao > melhor_valor:
                    melhor_valor = valor_acao
                    melhor_acao = acao
            politica[estado] = melhor_acao
            if acao_antiga != melhor_acao:
                estavel = False
    return politica, valores

# Execução e resultados
valores_otimos = iteracao_valor()
politica_otima_vi = obter_politica_gulosa(valores_otimos)
politica_otima_pi, valores_pi = iteracao_politica()

print("Valor ótimo (Iteração de Valor):", valores_otimos)
print("Política ótima (Iteração de Valor):", politica_otima_vi)
print("Política ótima (Iteração de Política):", politica_otima_pi)
print("Valores (Iteração de Política):", valores_pi)

Valor ótimo (Iteração de Valor): {'s1': 7.622070542910445, 's2': 6.774612915791802}
Política ótima (Iteração de Valor): {'s1': 'x12', 's2': 'x22'}
Política ótima (Iteração de Política): {'s1': 'x12', 's2': 'x22'}
Valores (Iteração de Política): {'s1': 7.629598945767491, 's2': 6.782141318648846}
